# Generate Dataframe scraping the Canada Postal Code from the [wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

<div class="alert alert-warning alertwarning" style="margin-top: 20px">
    
<h3>Libraries</h3>
pandas, urllib.request, BeautifulSoup, geocoder

<h3>Keywords</h3>
Scrapping, Check installed packages, Inner Join dataframes,
</div>

<a id="top"></a>
<div class="alert alert-block alert-info" style="margin-top: 20px">
<h1>Table of Contents</h1>
<hr>
<ol>
    <li><a href="#Part_1">Part 1 - Scrape data from Wikipedia</a></li>
    <li><a href="#Part_2">Part 2 - Combining two dataframes</a></li>
</ol>
<hr>
</div>


***

<div class="alert alert-warning alertwarning" style="margin-top: 20px">
<a id="Part_1"></a>
<h2>Part 1 - Scrape data from Wikipedia</h2>
    
<a href="#top">Top</a>
</div>

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

### The function GetCanadaPostalCode will scrape the postal code from the wikipedia webpage using the library [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#calling-a-tag-is-like-calling-find-all)

In [2]:
def searchPostalCode(postalcodes, postalcode):
    '''
    This function will search for the postalcode and return its position in the array
    If postcode not found it will return -1
    '''
    totalrows = len(postalcodes)
    # if totalrows is zero that means the array is empty, so no need search
    if totalrows==0:
        return -1
    
    totalcols = len(postalcodes[0]) # check the length of the first row, all rows should have the same number of columns
    
    #Loop through all thw postalcodes
    for x in range(totalrows):
        for y in range(totalcols):
            # Position zero is the postal code
            if postalcodes[x][0] == postalcodes:
                return x
    return -1

In [3]:
def GetCanadaPostalCode(baseurl):
    '''
    This function will return 1 array containing the column names and 2nd array containing all postalcodes 
    
    '''
    postalcodes = []
    
    #Read Web Page
    content = urllib.request.urlopen(baseurl).read()
    
     #---------------------------------------------
    #Scrape Table Header from Web Page
    column_names=[]
    for rows in BeautifulSoup(content, "lxml").findAll("table"):
        for index, cols in enumerate(rows.findAll("th")):
            cv = cols.get_text().rstrip("\n")
            column_names.append(cv)
        
        #We need just the first table, the page has two tables
        break
    
    #---------------------------------------------
    #Scrape Table Content from Web Page
    #find all tr tags
    #Loop through all the rows in the table
    for rows in BeautifulSoup(content, "lxml").findAll("tr"):
        
        #search inside each row all tag td
        totalcol = len(rows.findAll("td"))
        
        if totalcol==3:
            #initiate variables
            cn = 0
            col=[]
            addRow=True
            
            #Loop through all the columns in the table
            for index, cols in enumerate(rows.findAll("td")):
                #cleanup carriage return
                cv = cols.get_text().rstrip("\n")
                #cleanup blank values or "Not assigned"
                if (cv=='Not assigned' or cv=="") :
                    addRow=False
                #append value in the column array
                col.append(cv)
                
            if addRow==True:
                #Search of existing postalcode and if found it will append the neighborhood name in the found position
                pos = searchPostalCode(postalcodes,col[0][0])
                if (pos==-1) :
                    #append the new postalcode in the array
                    postalcodes.append(col)
                else:
                    #append the neighborhood name in the found position
                    postalcodes[pos][3]+=", "+col[0][2]

    return column_names, postalcodes

### Read data

In [4]:
Header, PostalCodes = GetCanadaPostalCode("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

### Generate dataframe

In [5]:
df_postalcodes = pd.DataFrame(PostalCodes, columns=Header) 

### Check results, sorting by column Postal code

In [6]:
df_postalcodes.sort_values(by='Postal Code', ascending=True).head(12)

,Postal Code,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
58,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [7]:
df_postalcodes.shape

(103, 3)

<div class="alert alert-warning alertwarning" style="margin-top: 20px">
<a id="Part_2"></a>
<h2>Part 2 - Combining two dataframes</h2>
    
<a href="#top">Top</a>
</div>

In [8]:
import sys
import subprocess
import pkg_resources

#check if the library is installed
required = {'geocoder'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)
else:
    print('Libraries are all set !')


Libraries are all set !


In [9]:
import geocoder

In [10]:
def getCoordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
        

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [11]:
postal_code = "M5G"
latitude , longitude = getCoordinates(postal_code)
print(f"Postal code {postal_code} (latitude {latitude}, longitude {longitude})")   

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h3>Results</h3>
<a href="#top">Top</a>

As you can see above, the api is not returning anything, I tried couple different times, so I will use the data coming from the repository 
</div>

### Download data from IBM Repository into local repository

In [12]:
#remove the file, if exist
!rm -rf 'toronto_data.csv'
#Download the geospatial_data into a local json file
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
!ls
print('Data downloaded!')

toronto_data.csv
Data downloaded!


### Read the downloaded file

In [13]:
df_geodata = pd.read_csv('toronto_data.csv', keep_default_na=False, na_values=[""])

In [14]:
print(df_geodata)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24          M2R  43.782736 -79.442259
25          

### Execute a inner join between df_postalcodes and df_geodata [datacarpentry reference](https://datacarpentry.org/python-ecology-lesson/05-merging-data/)

In [15]:
df_toronto = pd.merge(left=df_postalcodes, right=df_geodata, left_on='Postal Code', right_on='Postal Code')

### Display the first 20 records

In [16]:
df_toronto.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
